In [1]:
import warnings
warnings.simplefilter(action='ignore')

import pandas as pd
import numpy as np
import datetime
city_popu=pd.read_hdf('./Processed_Data/citypopu.h5',key='raw2')
city_popu['城市']=city_popu['城市'].str.replace('市',"")
city_popu['常住人口']=pd.to_numeric(city_popu['常住人口'])

import os
dfo=pd.DataFrame()

for f in os.listdir('./DataSource/baiduindex_binyiguan/'):
    print(f)
    d=pd.read_csv('./DataSource/baiduindex_binyiguan\%s' % f)
    dfo=pd.concat([dfo,d],axis=0)


baiduindex_2018-1225_2022-1225_2022-12-26.csv
baiduindex_2022-12-26_2022-12-28_20221229.csv
baiduindex_2022-12-29_2022-12-29_20221230.csv
baiduindex_2022-12-30_2023-01-03_20230104.csv


In [2]:
dfo=dfo.merge(city_popu,left_on='cityName',right_on='城市',how='left')
dfo

,searchKey,provinceName,cityName,startDate,endDate,all,pc,mobile,time,城市,常住人口
0,殡仪馆,贵州,贵阳,2021-12-25,2021-12-25,61.0,NaN,61.0,2022-12-26 16:12:21,贵阳,451
1,殡仪馆,贵州,贵阳,2021-12-26,2021-12-26,61.0,NaN,61.0,2022-12-26 16:12:21,贵阳,451
2,殡仪馆,贵州,贵阳,2021-12-27,2021-12-27,70.0,NaN,70.0,2022-12-26 16:12:21,贵阳,451
3,殡仪馆,贵州,贵阳,2021-12-28,2021-12-28,70.0,NaN,70.0,2022-12-26 16:12:21,贵阳,451
4,殡仪馆,贵州,贵阳,2021-12-29,2021-12-29,63.0,NaN,63.0,2022-12-26 16:12:21,贵阳,451
...,...,...,...,...,...,...,...,...,...,...,...
145229,殡仪馆,陕西,咸阳,2022-12-30,2022-12-30,130.0,57.0,73.0,2023-01-04 16:40:02,咸阳,80
145230,殡仪馆,陕西,咸阳,2022-12-31,2022-12-31,73.0,NaN,73.0,2023-01-04 16:40:02,咸阳,80
145231,殡仪馆,陕西,咸阳,2023-01-01,2023-01-01,58.0,NaN,58.0,2023-01-04 16:40:02,咸阳,80
145232,殡仪馆,陕西,咸阳,2023-01-02,2023-01-02,67.0,NaN,67.0,2023-01-04 16:40:02,咸阳,80


In [3]:
dfo['startDate']=pd.to_datetime(dfo['startDate'])

In [8]:
pd.pivot_table(dfo,index='cityName',columns='startDate',values='all',aggfunc='mean').to_clipboard()

In [9]:
c1=dfo['startDate']>'2022-11-01'
by=dfo[c1][['cityName','startDate','all','常住人口']]

# datelist=by['startDate'].drop_duplicates().sort_values()[-20:]

def get_status(x):
    now=float(x['now'])
    max=float(x['max'])
    per=now/max
    if x['startDate']<=x['max_date']:
        return '0 未达峰'
    else:
        if per >0.8:
            return '1 下降至 80-100%'
        if 0.5<per <=0.8:
            return '2 下降至 50-80%'
        if per <=0.5:
            return '3 下降至 50%以下'


# c2=by['startDate']=='2022-12-28'
d1=by.rename(columns={'all':'now'})
d2=by.groupby(by='cityName')['all'].max().reset_index().rename(columns={'all':'max'})
d2=d2.merge(d1.sort_values(by=['cityName','startDate']).drop_duplicates(subset=['cityName','now']),left_on=['cityName','max'],right_on=['cityName','now'],how='left').rename(columns={'startDate':'max_date'})
df=d1.merge(d2[['cityName', 'max', 'max_date']],on='cityName',how='left')

df['status']=df.apply(lambda x:get_status(x),axis=1)

pv_value=pd.pivot_table(df.drop_duplicates(), values='常住人口', index=['startDate'],columns=["status"], aggfunc=np.sum)
pv_value=pv_value.div(pv_value.sum(axis=1), axis=0)

pv_value['0 sum']=1
pv_value=pv_value.fillna(0)
pv_value=pv_value.sort_index(axis=1)

pv_value.tail(15).to_clipboard(header=None)